<a href="https://colab.research.google.com/github/peterbabulik/QuILT/blob/main/QuILT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import torch
import torch.nn as nn
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
import math

# ==============================================================================
# PART 1: The Mathematical Building Blocks (With Entanglement)
# ==============================================================================

def get_one_qubit_operator(gate_matrix, target_qubit, total_qubits):
    I = torch.eye(2, dtype=torch.cfloat)
    op_list = [I] * total_qubits
    op_list[target_qubit] = gate_matrix

    full_op = op_list[0]
    for i in range(1, total_qubits):
        full_op = torch.kron(full_op, op_list[i])
    return full_op

def get_cnot_operator(control_qubit, target_qubit, total_qubits):
    """
    Constructs the CNOT operator matrix. This is a more complex operation.
    """
    I = torch.eye(2, dtype=torch.cfloat)
    Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.cfloat)
    X = torch.tensor([[0, 1], [1, 0]], dtype=torch.cfloat)

    # CNOT can be decomposed into single-qubit gates and CZ
    # CNOT(c,t) = (I_c @ H_t) * CZ(c,t) * (I_c @ H_t)
    # The CZ gate is easier to construct with tensor products

    # Projector onto |0>
    P0x0 = torch.tensor([[1, 0], [0, 0]], dtype=torch.cfloat)
    # Projector onto |1>
    P1x1 = torch.tensor([[0, 0], [0, 1]], dtype=torch.cfloat)

    # Identity on all qubits
    Id_all = torch.eye(2**total_qubits, dtype=torch.cfloat)

    # Term 1: I...I_c...I_t...I
    term1_op_list = [I] * total_qubits
    term1_op_list[control_qubit] = P0x0

    term1 = term1_op_list[0]
    for i in range(1, total_qubits):
        term1 = torch.kron(term1, term1_op_list[i])

    # Term 2: I...X_c...X_t...I
    term2_op_list = [I] * total_qubits
    term2_op_list[control_qubit] = P1x1
    term2_op_list[target_qubit] = X

    term2 = term2_op_list[0]
    for i in range(1, total_qubits):
        term2 = torch.kron(term2, term2_op_list[i])

    return term1 + term2

# --- Define Gate Matrices (Differentiable) ---
def ry_matrix(theta):
    c = torch.cos(theta / 2)
    s = torch.sin(theta / 2)
    row1 = torch.stack([c, -s])
    row2 = torch.stack([s, c])
    return torch.stack([row1, row2]).to(torch.cfloat)

def rz_matrix(theta):
    angle = theta / 2.0
    diag_elements = torch.polar(torch.ones(2), torch.stack([-angle, angle]))
    return torch.diag(diag_elements)

Z_matrix = torch.tensor([[1, 0], [0, -1]], dtype=torch.cfloat)

# ==============================================================================
# PART 2: The Quantum AI Model with Entanglement
# ==============================================================================

class QuantumCircuitSimulator(nn.Module):
    def __init__(self, n_qubits, n_features, circuit_depth=4):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_features = n_features
        self.circuit_depth = circuit_depth
        self.params = nn.Parameter(torch.rand((circuit_depth, n_qubits, 2)) * 2 * math.pi)

    def forward(self, x_batch):
        predictions = []
        for x in x_batch:
            psi = torch.zeros(2**self.n_qubits, dtype=torch.cfloat)
            psi[0] = 1.0

            # Data Encoding
            for i in range(self.n_qubits):
                angle = x[i % self.n_features]
                op = get_one_qubit_operator(ry_matrix(angle), i, self.n_qubits)
                psi = op @ psi

            # Variational Circuit
            for d in range(self.circuit_depth):
                # Rotations
                for i in range(self.n_qubits):
                    op_ry = get_one_qubit_operator(ry_matrix(self.params[d, i, 0]), i, self.n_qubits)
                    psi = op_ry @ psi
                    op_rz = get_one_qubit_operator(rz_matrix(self.params[d, i, 1]), i, self.n_qubits)
                    psi = op_rz @ psi

                # --- ENTANGLEMENT LAYER ---
                for i in range(0, self.n_qubits - 1, 2):
                    op_cnot = get_cnot_operator(i, i+1, self.n_qubits)
                    psi = op_cnot @ psi
                for i in range(1, self.n_qubits - 1, 2):
                    op_cnot = get_cnot_operator(i, i+1, self.n_qubits)
                    psi = op_cnot @ psi

            # Measurement
            measurement_op = get_one_qubit_operator(Z_matrix, 0, self.n_qubits)
            op_psi = measurement_op @ psi
            prediction = torch.real(torch.vdot(psi, op_psi))
            predictions.append(prediction)

        return torch.stack(predictions)

# ==============================================================================
# PART 3: Data and Training
# ==============================================================================
X, y = make_circles(n_samples=100, factor=0.5, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

X_min, X_max = X_train.min(), X_train.max()
X_train = (X_train - X_min) / (X_max - X_min) * math.pi
X_test = (X_test - X_min) / (X_max - X_min) * math.pi

# --- HYPERPARAMETER TUNING ---
# We might need a deeper circuit or more qubits to solve this now.
n_qubits = 4
circuit_depth = 4 # Increased depth to give more room for learning
learning_rate = 0.05

model = QuantumCircuitSimulator(n_qubits=n_qubits, n_features=X_train.shape[1], circuit_depth=circuit_depth)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.BCEWithLogitsLoss()

print(f"--- Training Full Statevector model on {n_qubits} qubits with Entanglement ---")
epochs = 80 # More epochs may be needed

for epoch in range(epochs):
    optimizer.zero_grad()
    logits = model(X_train)
    loss = loss_fn(logits.squeeze(), y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

with torch.no_grad():
    test_logits = model(X_test)
    test_preds = torch.sigmoid(test_logits.squeeze()) > 0.5
    accuracy = (test_preds.float() == y_test).float().mean()
    print(f"\nFinal Test Accuracy: {accuracy.item() * 100:.2f}%")

--- Training Full Statevector model on 4 qubits with Entanglement ---
Epoch 10, Loss: 0.6518
Epoch 20, Loss: 0.6030
Epoch 30, Loss: 0.5673
Epoch 40, Loss: 0.5518
Epoch 50, Loss: 0.5471
Epoch 60, Loss: 0.5465
Epoch 70, Loss: 0.5459
Epoch 80, Loss: 0.5457

Final Test Accuracy: 86.67%
